In [2]:
import xarray as xr
import rasterio
from rasterio.transform import Affine
from rasterio.warp import calculate_default_transform, reproject, Resampling

# --- User inputs ---
netcdf_path = r"E:\GOES-R Lightning Data\WEST-2025\OR_GLM-L3-GLMF-M6_G18_s202518123590000_e202518200000000_c20251820001000.nc"
raster_path = r"E:\GOES-R Lightning Data\WEST-RASTERS\west_max_energy_2025_cleaned.tif"
output_raster = r"E:\GOES-R Lightning Data\WEST-RASTERS\west_max_energy_2025_georef.tif"
reproj_raster = r"E:\GOES-R Lightning Data\WEST-RASTERS\west_max_energy_2025_wgs84.tif"

#netcdf_path = r"E:\GOES-R Lightning Data\EAST-2025\OR_GLM-L3-GLMF-M6_G19_s202518200100000_e202518200110000_c20251820012050.nc"
#raster_path = r"E:\GOES-R Lightning Data\EAST-RASTERS\east_max_energy_2025_cleaned.tif"
#output_raster = r"E:\GOES-R Lightning Data\EAST-RASTERS\east_max_energy_2025_georef.tif"
#reproj_raster = r"E:\GOES-R Lightning Data\EAST-RASTERS\east_max_energy_2025_wgs84.tif"

# --- 1. Open NetCDF ---
ds = xr.open_dataset(netcdf_path)

# GLM variable of interest
var_name = 'Total_Optical_energy'
da_nc = ds[var_name]

# Projection metadata
proj_var = ds['goes_imager_projection']
h = proj_var.perspective_point_height         # satellite height
lon_0 = proj_var.longitude_of_projection_origin
sweep = proj_var.sweep_angle_axis
a = proj_var.semi_major_axis
b = proj_var.semi_minor_axis

# --- 2. Build CRS string ---
crs_string = (
    f"+proj=geos +h={h} +lon_0={lon_0} +sweep={sweep} "
    f"+a={a} +b={b} +units=m +no_defs"
)

# --- 3. Convert x/y to meters ---
x = ds['x'].values  # in radians
y = ds['y'].values  # in radians

x_m = x * h
y_m = y * h

# --- 4. Build affine transform ---
pixel_width = x_m[1] - x_m[0]
pixel_height = y_m[1] - y_m[0]

x_min = x_m[0] - pixel_width / 2
y_max = y_m[0] - pixel_height / 2

transform = Affine(pixel_width, 0, x_min, 0, pixel_height, y_max)

# --- 5. Open existing GeoTIFF ---
with rasterio.open(raster_path, 'r') as src:
    data = src.read(1)
    profile = src.profile

# --- 6. Assign CRS and transform ---
profile.update({
    'crs': crs_string,
    'transform': transform
})

# --- 7. Save georeferenced GeoTIFF ---
with rasterio.open(output_raster, 'w', **profile) as dst:
    dst.write(data, 1)

print(f"Saved georeferenced raster to: {output_raster}")

# --- 8. Reproject to EPSG:4326 ---
dst_crs = 'EPSG:4326'
with rasterio.open(output_raster) as src:
    transform_new, width_new, height_new = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds
    )
    profile_new = src.profile.copy()
    profile_new.update({
        'crs': dst_crs,
        'transform': transform_new,
        'width': width_new,
        'height': height_new
    })

    data_new = rasterio.open(reproj_raster, 'w', **profile_new)
    reproject(
        source=rasterio.band(src, 1),
        destination=rasterio.band(data_new, 1),
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform_new,
        dst_crs=dst_crs,
        resampling=Resampling.nearest
    )
    data_new.close()

print(f"Saved reprojected raster to: {reproj_raster}")


Saved georeferenced raster to: E:\GOES-R Lightning Data\WEST-RASTERS\west_max_energy_2025_georef.tif
Saved reprojected raster to: E:\GOES-R Lightning Data\WEST-RASTERS\west_max_energy_2025_wgs84.tif
